In [1]:
import glob
import tensorflow as tf
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import keras as k

Using TensorFlow backend.


In [2]:
data = pd.read_csv("stroke.csv", index_col=0)

In [3]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [4]:
data = data.drop("ever_married" , axis=1)

In [5]:
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,Male,67.0,0,1,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Self-employed,Rural,174.12,24.0,never smoked,1


In [6]:
column_req = ["gender", "age", "hypertension", "work_type", "Residence_type", "avg_glucose_level", "bmi", "smoking_status","stroke"]

In [7]:
for column in data.columns:
    if data[column].dtype == np.number:
        continue
    data[column] = LabelEncoder().fit_transform( data[column] )

In [8]:
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.6,1,1
51676,0,61.0,0,0,3,0,202.21,NaN,2,1
31112,1,80.0,0,1,2,0,105.92,32.5,2,1
60182,0,49.0,0,0,2,1,171.23,34.4,3,1
1665,0,79.0,1,0,3,0,174.12,24.0,2,1


In [9]:
data = data.fillna(data.mean())
data.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.600000,1,1
51676,0,61.0,0,0,3,0,202.21,28.893237,2,1
31112,1,80.0,0,1,2,0,105.92,32.500000,2,1
60182,0,49.0,0,0,2,1,171.23,34.400000,3,1
1665,0,79.0,1,0,3,0,174.12,24.000000,2,1


In [10]:
X = data.drop(["stroke"], axis=1)
Y = data["stroke"]

In [11]:
X.head()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,
9046,1,67.0,0,1,2,1,228.69,36.600000,1
51676,0,61.0,0,0,3,0,202.21,28.893237,2
31112,1,80.0,0,1,2,0,105.92,32.500000,2
60182,0,49.0,0,0,2,1,171.23,34.400000,3
1665,0,79.0,1,0,3,0,174.12,24.000000,2


In [12]:
Y.head()


id
9046     1
51676    1
31112    1
60182    1
1665     1
Name: stroke, dtype: int64

In [13]:
x_scaler = MinMaxScaler()
x_scaler.fit(X)
column_names = X.columns
X[column_names]=x_scaler.transform(X)

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y, test_size= 0.2, shuffle = True)

In [15]:
print("Before OverSampling, counts of label '1': {}".format(sum(Y_train == 1))) 
print("Before OverSampling, counts of label '0': {} \n".format(sum(Y_train == 0))) 

Before OverSampling, counts of label '1': 197
Before OverSampling, counts of label '0': 3891 



In [16]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_train_res, Y_train_res = sm.fit_sample(X_train, Y_train.ravel()) 


In [17]:
print("After OverSampling, counts of label '1': {}".format(sum(Y_train_res == 1))) 
print("After OverSampling, counts of label '0': {}".format(sum(Y_train_res == 0))) 

Y_test.head()

After OverSampling, counts of label '1': 3891
After OverSampling, counts of label '0': 3891


id
36431    0
72082    0
52361    0
46647    0
31112    1
Name: stroke, dtype: int64

# Build model

In [18]:
model = k.Sequential()

In [22]:
model.add(k.layers.Dense(10,activation=tf.nn.relu,input_shape=(X_train_res.shape[1],)))
model.add(k.layers.Dense(64,activation=tf.nn.relu))
model.add(k.layers.Dense(32,activation=tf.nn.relu))
model.add(k.layers.Dense(32,activation=tf.nn.relu))
model.add(k.layers.Dense(16,activation=tf.nn.relu))
model.add(k.layers.Dense(1,activation =tf.nn.sigmoid))

# Compiling the model

In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train the model

In [24]:
history = model.fit(X_train_res, Y_train_res, epochs = 2000, batch_size = X_train_res.shape[0])

score, acc = model.evaluate(X_test, Y_test,
                            batch_size=X_test.shape[0])
print('Test score:', score)
print('Test accuracy:', acc)

Epoch 1/2000
7782/7782 [==============================] - 1s 109us/step - loss: 0.6911 - accuracy: 0.8517
Epoch 2/2000
7782/7782 [==============================] - 0s 4us/step - loss: 0.6875 - accuracy: 0.5000
Epoch 3/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.6838 - accuracy: 0.5000
Epoch 4/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.6788 - accuracy: 0.5000
Epoch 5/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.6749 - accuracy: 0.5000
Epoch 6/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.6727 - accuracy: 0.5000
Epoch 7/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.6678 - accuracy: 0.8958
Epoch 8/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.6647 - accuracy: 0.8894
Epoch 9/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.6598 - accuracy: 0.9243
Epoch 10/2000
7782/7782 [==============================] - 0s 

Epoch 79/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2136 - accuracy: 0.9616
Epoch 80/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2076 - accuracy: 0.9617
Epoch 81/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2022 - accuracy: 0.9631
Epoch 82/2000
7782/7782 [==============================] - 0s 4us/step - loss: 0.1992 - accuracy: 0.9600
Epoch 83/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1969 - accuracy: 0.9590
Epoch 84/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2048 - accuracy: 0.9523
Epoch 85/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1958 - accuracy: 0.9539
Epoch 86/2000
7782/7782 [==============================] - 0s 4us/step - loss: 0.1951 - accuracy: 0.9534
Epoch 87/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1761 - accuracy: 0.9625
Epoch 88/2000
7782/7782 [==============================

Epoch 157/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1147 - accuracy: 0.9666
Epoch 158/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1147 - accuracy: 0.9668
Epoch 159/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1148 - accuracy: 0.9665
Epoch 160/2000
7782/7782 [==============================] - 0s 4us/step - loss: 0.1154 - accuracy: 0.9658
Epoch 161/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1160 - accuracy: 0.9656
Epoch 162/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1194 - accuracy: 0.9625
Epoch 163/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1189 - accuracy: 0.9641
Epoch 164/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1260 - accuracy: 0.9591
Epoch 165/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1172 - accuracy: 0.9648
Epoch 166/2000
7782/7782 [====================

Epoch 235/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2483 - accuracy: 0.9048
Epoch 236/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2449 - accuracy: 0.9108
Epoch 237/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2397 - accuracy: 0.9176
Epoch 238/2000
7782/7782 [==============================] - 0s 4us/step - loss: 0.2330 - accuracy: 0.9185
Epoch 239/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2250 - accuracy: 0.9201
Epoch 240/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2190 - accuracy: 0.9221
Epoch 241/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2134 - accuracy: 0.9255
Epoch 242/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2025 - accuracy: 0.9329
Epoch 243/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1990 - accuracy: 0.9314
Epoch 244/2000
7782/7782 [====================

Epoch 313/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1081 - accuracy: 0.9688
Epoch 314/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1081 - accuracy: 0.9685
Epoch 315/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1082 - accuracy: 0.9685
Epoch 316/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1088 - accuracy: 0.9681
Epoch 317/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1092 - accuracy: 0.9681
Epoch 318/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1117 - accuracy: 0.9658
Epoch 319/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1106 - accuracy: 0.9668
Epoch 320/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1123 - accuracy: 0.9649
Epoch 321/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1089 - accuracy: 0.9683
Epoch 322/2000
7782/7782 [====================

...

Epoch 1631/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2708 - accuracy: 0.8962
Epoch 1632/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2678 - accuracy: 0.8977
Epoch 1633/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2644 - accuracy: 0.8987
Epoch 1634/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2608 - accuracy: 0.9008
Epoch 1635/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2571 - accuracy: 0.9034
Epoch 1636/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2536 - accuracy: 0.9050
Epoch 1637/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2499 - accuracy: 0.9076
Epoch 1638/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2465 - accuracy: 0.9098
Epoch 1639/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2429 - accuracy: 0.9116
Epoch 1640/2000
7782/7782 [==========

Epoch 1708/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1167 - accuracy: 0.9663
Epoch 1709/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1165 - accuracy: 0.9662
Epoch 1710/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1162 - accuracy: 0.9666
Epoch 1711/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1160 - accuracy: 0.9672
Epoch 1712/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1157 - accuracy: 0.9674
Epoch 1713/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1155 - accuracy: 0.9675
Epoch 1714/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1153 - accuracy: 0.9671
Epoch 1715/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1151 - accuracy: 0.9670
Epoch 1716/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1149 - accuracy: 0.9672
Epoch 1717/2000
7782/7782 [==========

Epoch 1785/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1097 - accuracy: 0.9683
Epoch 1786/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1106 - accuracy: 0.9680
Epoch 1787/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1128 - accuracy: 0.9659
Epoch 1788/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1114 - accuracy: 0.9675
Epoch 1789/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1144 - accuracy: 0.9654
Epoch 1790/2000
7782/7782 [==============================] - 0s 7us/step - loss: 0.1113 - accuracy: 0.9675
Epoch 1791/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1092 - accuracy: 0.9689
Epoch 1792/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1084 - accuracy: 0.9692
Epoch 1793/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1088 - accuracy: 0.9699
Epoch 1794/2000
7782/7782 [==========

Epoch 1862/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2948 - accuracy: 0.8899
Epoch 1863/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2852 - accuracy: 0.8901
Epoch 1864/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2863 - accuracy: 0.8852
Epoch 1865/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2828 - accuracy: 0.8878
Epoch 1866/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2739 - accuracy: 0.8971
Epoch 1867/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2767 - accuracy: 0.8987
Epoch 1868/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.2719 - accuracy: 0.9007
Epoch 1869/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2673 - accuracy: 0.9023
Epoch 1870/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.2687 - accuracy: 0.8996
Epoch 1871/2000
7782/7782 [==========

Epoch 1939/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1141 - accuracy: 0.9677
Epoch 1940/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1135 - accuracy: 0.9675
Epoch 1941/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1130 - accuracy: 0.9676
Epoch 1942/2000
7782/7782 [==============================] - 0s 7us/step - loss: 0.1126 - accuracy: 0.9679
Epoch 1943/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1122 - accuracy: 0.9686
Epoch 1944/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1118 - accuracy: 0.9695
Epoch 1945/2000
7782/7782 [==============================] - 0s 6us/step - loss: 0.1113 - accuracy: 0.9693
Epoch 1946/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1110 - accuracy: 0.9695
Epoch 1947/2000
7782/7782 [==============================] - 0s 5us/step - loss: 0.1106 - accuracy: 0.9697
Epoch 1948/2000
7782/7782 [==========

In [42]:
model.save('ckd.model')

In [43]:
print("shape of train data", X_train_res.shape)
print("shape of test data", X_test.shape)

shape of train data (7796, 9)
shape of test data (1022, 9)


Cell for user input